In [ ]:
#!pip install schedule
#!pip install playsound
#!pip install pygame

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from scrapingant_client import ScrapingAntClient
from playsound import playsound  # Import the playsound library
import re
import csv
import aiohttp
import asyncio
import nest_asyncio
from aiohttp import ClientSession
import time
import schedule
from datetime import datetime
import os

nest_asyncio.apply()
#Works

def run_scraping_job():

    base_url = 'https://www.webuycars.co.za/buy-a-car/'
    #base_image='https://photos.webuycars.co.za/photobooth/{}/Images/{}.webp'

    carslist = pd.read_csv('cars_list.csv', header=None).iloc[:, 0].tolist()
    #carslist= ['8B7605068','C2EC07194','087E12450','567204394','E82106689','AF8C14308','DFCE12681','851500707','37E903562','567204394']

    csv_file_name = 'cars_list.csv'

    # Function to write the list to a CSV file
    def write_to_csv(data_list):
        with open(csv_file_name, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            for item in data_list:
                csv_writer.writerow([item])

                # Set the timeout duration in seconds (1 minute in this case)
    timeout_duration = 30

    # Record the start time
    input_time = time.time()

    def play_sound():
    # Replace 'path_to_your_sound_file.mp3' with the actual path to your sound file
        playsound('sound.mp3')
    
    while True:
        # Ask for user input
        elapsed_time = time.time() - input_time
        
        if elapsed_time >= timeout_duration:
            print("30 Seconds passed without input. Breaking out of the loop.")
            break
        
        play_sound() #before needing user input 
        user_input = input("Enter a car ID (enter '#' to skip): ")
           
        # Check if user entered '#', then skip
        if user_input == '#':
            break

        # Check if user input is the same length as other items in the carslist
        if all(len(user_input) == len(car) for car in carslist):
            # Check if user input is in the carslist
            if user_input not in carslist:
                # If not, add it to the list
                carslist.append(user_input)
                print(f"Car ID {user_input} added to the list.")
                 # Write the updated carslist to the CSV file
                write_to_csv(carslist)

            else:
                print(f"Car ID {user_input} is already in the list.")

            # Print the updated carslist
            print("Updated carslist:", carslist)
            break
        else:
            print("Error: The length of the entered car ID does not match the existing car IDs. Please enter a valid car ID.")

    car_urls = [f'{base_url}{car_id}/' for car_id in carslist]

    # Create a ScrapingAntClient instance
    #client = ScrapingAntClient(token='d0f560d5a3e540999d91f2c73fe5c98e') #0bddd99558144477a27556352aafc386
    client = ScrapingAntClient(token='0bddd99558144477a27556352aafc386') #0bddd99558144477a27556352aafc386

    page_contents = []
    print("skipped")
    # Get the HTML page rendered content
    #page_content = client.general_request(url).content
    all_contents = []
    cache= {} #cache
    #read carslist again 
    carslist = pd.read_csv('cars_list.csv', header=None).iloc[:, 0].tolist()

    def save_to_file(car_id, content):  #cache
        with open(f'{car_id}.html', 'wb') as file:
            file.write(content)
            print(f"Cached content saved to file for car {car_id}")
    
    #Works
    
    async def fetch_page_content(car_id):
        car_url = f'{base_url}{car_id}/'
        page_content = client.general_request(car_url).content
        return page_content

    async def main():

        # Create an aiohttp session with increased connector limit
        connector = aiohttp.TCPConnector(limit_per_host=20)
        async with aiohttp.ClientSession() as session:
            # Use asyncio.gather to fetch page_content concurrently
            tasks = [fetch_page_content(car_id) for car_id in carslist]
            page_contents = await asyncio.gather(*tasks)

        # Print the length of each page_content
        for i, content in enumerate(page_contents):
              #create Cache
            # Cache the content
            #cache[carslist[i]] = content  #cache

            # Save cached content to a file
            #save_to_file(carslist[i], cache[carslist[i]])  #cache
            
            print(f"Length of page_content for car {carslist[i]}: {len(content)}")

        for content in page_contents:
            all_contents.append(content)

    start_time = time.time()
    asyncio.run(main())
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Elapsed time: {elapsed_time} seconds")

    #if __name__ == "__main__":
    #    asyncio.run(main())

    #modified to make DF for all pages 

    def scrape_website(page_content):
        # Send an HTTP request to the URL       
        # Check if the request was successful (status code 200)
        if page_content is not None:
            # Parse the HTML content of the page
            soup = BeautifulSoup(page_content)

            # Extract information based on specified classes and selectors
         # Extract information based on specified classes and selectors
            price = soup.select_one('.car-price div:nth-of-type(2) span:nth-of-type(1)')
            if price is not None:
                price = price.text.strip()
                price = re.sub(r'\D', '', price)
            Auction_price = soup.select_one('.wheel-text-middle span')
            if Auction_price is not None:
                Auction_price = Auction_price.text.strip()
                Auction_price = re.sub(r'\D', '', Auction_price) 

            name_element = soup.select_one('div.car-description')
            name = name_element.text.strip() if name_element is not None else None

            branch_element = soup.select_one('tr:contains("Branch:") td.tab-content')
            branch = branch_element.text.strip() if branch_element is not None else None

            mileage_element = soup.select_one('tr:contains("Mileage:") td.tab-content')
            mileage = mileage_element.text.strip() if mileage_element is not None else None

            service_history_element = soup.select_one('tr:contains("Service history:") td.tab-content')
            service_history = service_history_element.text.strip() if service_history_element is not None else None

            consumption_element = soup.select_one('tr:contains("Fuel consumption:") td.tab-content')
            consumption = consumption_element.text.strip() if consumption_element is not None else None

            fuel_element = soup.select_one('tr:contains("Fuel type:") td.tab-content')
            fuel = fuel_element.text.strip() if fuel_element is not None else None

            stock_code_element = soup.select_one('tr:contains("Stock code:") td.tab-content')
            stock_code = stock_code_element.text.strip() if stock_code_element is not None else None

            current_datetime = datetime.now()
            current_date = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    
            # Create a list of dictionaries with the extracted information
            data = {
                'Name': name,
                'Price': price,
                'Auction Price': Auction_price,
                'Branch': branch,
                'Mileage': mileage,
                'Service History': service_history,
                'Fuel Consumption': consumption,
                'Fuel Type': fuel,
                'Stock Code':stock_code,
                'Date':current_date,
                'image':f'https://photos.webuycars.co.za/photobooth/{stock_code}/Images/{stock_code}.webp'
            }

            # Create a DataFrame using pandas
            #df = pd.DataFrame(data)
            return data

    all_data =[] 

    for content in all_contents:
        result_df = scrape_website(content)

        if result_df is not None:
            # Append the data from the current page to the list
            all_data.append(result_df)

    #Create a DataFrame using pandas with data from all pages
    df = pd.DataFrame(all_data)

    # Print the resulting DataFrame
    #print(df)
    Data = pd.DataFrame(all_data)
    #Write to CSV 
    #Apend Data
    
    now = datetime.now()
    current_date = now.strftime("%Y_%m_%d_%H_%M")
    #csv_file_path = f"{current_date}.csv"
    csv_file_path = "jan.csv"

    # Check if the CSV file exists
    if os.path.exists(csv_file_path):
        # Append to the existing file without writing headers
        Data.to_csv(csv_file_path, mode='a', header=False, index=False)
    else:
        # Create a new file and write headers
        Data.to_csv(csv_file_path, mode='w', header=True, index=False)

#schedule.every().hour.do(run_scraping_job) #1 hour
schedule.every(10).minutes.do(run_scraping_job) #10 minutes

while True:
    schedule.run_pending()
    time.sleep(1)

Enter a car ID (enter '#' to skip): #
skipped


TypeError: a bytes-like object is required, not 'str'

In [ ]:
csv_file_path